In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
DATA_PATH = ""
df = pd.read_csv(DATA_PATH, encoding='utf-8-sig')

Mounted at /content/drive


In [ ]:
!pip install -q "transformers>=4.40,<5" "peft>=0.11.0" accelerate scikit-learn tqdm

import os, re, gc, random, math, numpy as np, pandas as pd, torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm.notebook import tqdm
from contextlib import nullcontext

# 메모리 파편화 완화
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 시드/디바이스
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device, " | Torch:", torch.__version__)

# L100 최적화: TF32 + (기본) BF16
USE_BF16 = True
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.set_float32_matmul_precision("high")

def autocast_ctx():
    if not torch.cuda.is_available():
        return nullcontext()
    # BF16 미지원일 때 자동으로 FP16으로 폴백
    try:
        dtype = torch.bfloat16 if USE_BF16 else torch.float16
        return torch.autocast(device_type="cuda", dtype=dtype)
    except TypeError:
        # 드물게 환경에서 BF16 타입 지원이 불완전할 수 있음
        return torch.autocast(device_type="cuda", dtype=torch.float16)

# 하이퍼(속도/성능 트레이드오프: ★표는 속도에 큰 영향)
BASE_MODEL   = "monologg/koelectra-base-v3-discriminator"
MAX_LEN      = 192             # ★ 192~224 (줄수록 빠름)
BATCH_BAGS   = 8               # bag=문서, 배치 크기
EPOCHS       = 3

LR_ENCODER   = 2e-5
LR_HEAD      = 2e-4
WEIGHT_DECAY = 0.01

USE_GC       = False           # ★ Gradient Checkpointing 해제(빠름, 메모리 충분할 때)
VAL_BATCH1   = True            # 검증 배치=1로 OOM 방지
USE_COMPILE  = False           # torch.compile 시 True (첫 스텝 오버헤드 있음)

# Hard-positive 캐시
HARD_CACHE_TOPK = 2         # 문서별 항상 포함할 상위 의심 문단 수
REFRESH_HARD_EVERY_EPOCH = True

# MIL/로스 강화
POOL_MODE   = "lse"         # 'attn'|'lse'|'max'|'mean'  (lse가 예리)
LSE_TAU     = 12.0          # (기존 10) 예리함 조금 ↑
BAG_AGG     = "noisy_or"    # 'linear' or 'noisy_or'  (리콜↑)
INST_TOPK   = 2             # (기존 1) 양성 문서에서 2개 문단을 1로 당김
INST_LAMBDA = 0.15          # (기존 0.1) 보조손실 가중치 ↑

DIR = ''
OUTPUT_DIR = ''

Device: cuda  | Torch: 2.8.0+cu126


In [ ]:
# =========================================
# 1) 데이터 로딩 & 전처리
# =========================================
def minimal_preprocess(text):
    text = text.strip()
    text = re.sub(r'[\u4E00-\u9FFF]', '', text)                   # 한자 제거
    text = re.sub(r'<[^>]+>', '', text)                           # HTML 태그 제거
    text = re.sub(r'\(\s*[^\w가-힣]*\s*\)', '', text)             # 빈 괄호 제거
    text = re.sub(r'\([^\(\)]{0,20}[\?\~]{1,3}[^\(\)]{0,20}\)', '', text)  # ( ? ~ ? ) 제거
    text = re.sub(r'[.,]{3,}', '.', text)                         # ... → .
    text = re.sub(r'[()]{2,}', '', text)                          # 괄호 잔재 정리
    text = re.sub(r',\s*,+', ',', text)
    text = re.sub(r'\s+', ' ', text)                              # 중복 공백 제거
    return text
def split_paragraphs(text: str):
    """빈 줄 기준으로 문단 분할"""
        # 문단 기준: 두 줄 개행 우선
    paragraphs = [p.strip() for p in text.split('\n') if p.strip()]
    # # 만약 너무 적게 쪼개졌으면 한 줄 개행으로 재시도
    if len(paragraphs) <= 1:
        paragraphs = [p.strip() for p in text.split('\n') if p.strip()]
    return paragraphs
# 문단 분해
def split_paragraphs(text: str):
    if not isinstance(text, str):
        text = str(text)
    parts = [p.strip() for p in re.split(r'\n\s*', text) if p.strip()]
    return parts if parts else [text.strip()]



df = pd.read_csv(DIR+"train.csv")

#df["full_text"] = df["full_text"].apply(minimal_preprocess)
# 1) 문단 길이(문자 수) 계산
df['char_len'] = df['full_text'].str.len()

# 2) 라벨(generated)별 35 % · 95 % 퍼센타일 계산
percentiles = (
    df
      .groupby('generated')['char_len']
      .quantile([0.35, 0.95])        # 두 지점 한 번에 구함
      .unstack(level=1)              # 보기 편하게: index=라벨, columns=p35/p95
      .rename(columns={0.35: 'p35', 0.95: 'p95'})
)

print("라벨별 문단 길이 퍼센타일")
print(percentiles)
# 3) 위 기준을 이용해 필터링
mask = df.apply(
    lambda r: percentiles.loc[r['generated'], 'p35'] <= r['char_len'] <= percentiles.loc[r['generated'], 'p95'],
    axis=1
)

df1 = (
    df[mask]
      .reset_index(drop=True)
      .drop(columns=['char_len'])   # 길이 컬럼이 필요 없으면 제거
)


df1['paragraphs'] = df1['full_text'].apply(split_paragraphs)
df1 = df1.drop(columns=['full_text'])
train_df = df1.explode('paragraphs').rename(columns={'paragraphs':'full_text'}).reset_index(drop=True)
assert set(["title","full_text","generated"]).issubset(train_df.columns), "train.csv columns mismatch"

# title 기준 누수 방지 분할
gss = GroupShuffleSplit(n_splits=1, test_size=0.1, random_state=SEED)
tr_idx, va_idx = next(gss.split(train_df, groups=train_df["title"]))
tr_df = train_df.iloc[tr_idx].reset_index(drop=True)
va_df = train_df.iloc[va_idx].reset_index(drop=True)


print("Train docs:", len(tr_df), " | Valid docs:", len(va_df))

라벨별 문단 길이 퍼센타일
              p35     p95
generated                
0          1057.0  6999.0
1          1052.0  6873.3
Train docs: 673411  | Valid docs: 74577


In [ ]:

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [ ]:
#Dataset & Collate (★속도 핵심: 문단 서브샘플링)

# 불균형 가중치(문서 단위)
num_pos = int(tr_df["generated"].sum())
num_neg = len(tr_df) - num_pos
POS_WEIGHT = (num_neg / max(num_pos, 1))
print("pos_weight =", POS_WEIGHT)

class MILDataset(Dataset):
    def __init__(self, df, has_label=True):
        self.docs = []
        for _, r in df.iterrows():
            paras = split_paragraphs(r["full_text"]) if "full_text" in r else [r.get("paragraph_text","")]
            item = {"title": r["title"], "paras": paras}
            if has_label:
                item["label"] = int(r["generated"])
            self.docs.append(item)
    def __len__(self): return len(self.docs)
    def __getitem__(self, idx): return self.docs[idx]

tr_set = MILDataset(tr_df, has_label=True)
va_set = MILDataset(va_df, has_label=True)

# ★훈련용: 문서당 문단 K개만 랜덤 샘플(큰 속도 향상)
TRAIN_PARAS_PER_DOC = 12     # 6~12 추천. 늘릴수록 느리지만 성능↑
MAX_PARAS_CAP       = 64    # 아주 긴 문서 안전 상한

# 양성 가중치 강화
POS_WEIGHT  = (num_neg / max(num_pos, 1)) * 1.3   # 기존 비율에 1.3배 (리콜 지향)
USE_ASYM_FOCAL = True     # 비대칭 포컬로스 on
GAMMA_POS = 0.0           # 양성 쪽은 focusing 약하게
GAMMA_NEG = 2.0           # 음성 쪽은 강하게 억제
EPS = 1e-8

def collate_train_subsample(batch, tokenizer, max_len=256, k=TRAIN_PARAS_PER_DOC):
    all_texts, bag_bounds, titles, labels = [], [], [], []
    cursor = 0
    for d in batch:
        titles.append(d["title"])
        paras = d["paras"] if d["paras"] else [""]

        # 상한-후 서브샘플
        if len(paras) > MAX_PARAS_CAP:
            idx = np.random.choice(len(paras), MAX_PARAS_CAP, replace=False)
            paras = [paras[i] for i in sorted(idx)]
        if len(paras) > k:
            idx = np.random.choice(len(paras), k, replace=False)
            paras = [paras[i] for i in sorted(idx)]

        start = cursor
        all_texts.extend(paras)
        cursor += len(paras); end = cursor
        bag_bounds.append((start, end))
        labels.append(d["label"])
    enc = tokenizer(all_texts, truncation=True, padding=True, max_length=max_len, return_tensors="pt")
    return {
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "bag_bounds": torch.tensor(bag_bounds, dtype=torch.long),
        "titles": titles,
        "labels": torch.tensor(labels, dtype=torch.float32)
    }

def collate_eval_full(batch, tokenizer, max_len=256):
    # 검증은 전체 문단 사용(정확평가)
    all_texts, bag_bounds, titles, labels = [], [], [], []
    cursor = 0
    for d in batch:
        titles.append(d["title"])
        paras = d["paras"] if d["paras"] else [""]
        start = cursor
        all_texts.extend(paras)
        cursor += len(paras); end = cursor
        bag_bounds.append((start, end))
        labels.append(d["label"])
    enc = tokenizer(all_texts, truncation=True, padding=True, max_length=max_len, return_tensors="pt")
    return {
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "bag_bounds": torch.tensor(bag_bounds, dtype=torch.long),
        "titles": titles,
        "labels": torch.tensor(labels, dtype=torch.float32)
    }
def collate_train_hardmix(batch, tokenizer, max_len=256, k=TRAIN_PARAS_PER_DOC):
    all_texts, bag_bounds, titles, labels = [], [], [], []
    cursor = 0
    for d in batch:
        titles.append(d["title"])
        paras = d["paras"] if d["paras"] else [""]
        # 캡
        if len(paras) > MAX_PARAS_CAP:
            idx = np.random.choice(len(paras), MAX_PARAS_CAP, replace=False)
            paras = [paras[i] for i in sorted(idx)]

        must = hard_cache.get(d["title"], [])
        remain_idx = [i for i in range(len(paras)) if i not in must]
        need = max(0, min(k, len(paras)) - len(must))
        extra = np.random.choice(remain_idx, size=need, replace=False).tolist() if need>0 else []
        sel = sorted(must + extra)
        paras = [paras[i] for i in sel]

        start = cursor
        all_texts.extend(paras)
        cursor += len(paras); end = cursor
        bag_bounds.append((start, end))
        labels.append(d["label"])

    enc = tokenizer(all_texts, truncation=True, padding=True, max_length=max_len, return_tensors="pt")
    return {
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "bag_bounds": torch.tensor(bag_bounds, dtype=torch.long),
        "titles": titles,
        "labels": torch.tensor(labels, dtype=torch.float32)
    }

tr_loader = DataLoader(
    tr_set, batch_size=BATCH_BAGS, shuffle=True,
    collate_fn=lambda b: collate_train_hardmix(b, tokenizer, MAX_LEN),
    num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2
)
va_loader = DataLoader(
    va_set, batch_size=(1 if VAL_BATCH1 else BATCH_BAGS), shuffle=False,
    collate_fn=lambda b: collate_eval_full(b, tokenizer, MAX_LEN),
    num_workers=2, pin_memory=True, persistent_workers=True, prefetch_factor=2
)


pos_weight = 10.949021417037812


In [ ]:
# 모델 (LoRA + MILPooler + 보조손실 + 가중치BCE)
from transformers.models.electra import ElectraModel
from peft import LoraConfig, get_peft_model

class MILPooler(nn.Module):
    """Pooling 선택지: 'attn' | 'lse' | 'max' | 'mean'"""
    def __init__(self, d, mode="lse", r=128, lse_tau=10.0):
        super().__init__()
        self.mode = mode; self.tau = lse_tau
        if mode == "attn":
            self.W = nn.Linear(d, r)
            self.u = nn.Linear(r, 1, bias=False)
    def forward(self, H):  # H: [n_i, d]
        if self.mode == "mean":
            z = H.mean(dim=0)
            alpha = torch.full((H.size(0),), 1.0/H.size(0), device=H.device)
            return z, alpha
        if self.mode == "max":
            idx = torch.argmax(H.norm(dim=1))
            z = H[idx]
            alpha = torch.zeros(H.size(0), device=H.device); alpha[idx] = 1.0
            return z, alpha
        if self.mode == "lse":
            e = H.norm(dim=1) * self.tau           # τ↑ ⇒ max에 가까워짐
            alpha = torch.softmax(e, dim=0)
            z = torch.sum(alpha.unsqueeze(-1) * H, dim=0)
            return z, alpha
        # attention
        A = torch.tanh(self.W(H))
        e = self.u(A).squeeze(-1)
        alpha = torch.softmax(e, dim=0)
        z = torch.sum(alpha.unsqueeze(-1) * H, dim=0)
        return z, alpha

def instance_aux_loss(s_logits, alpha, y, k=1):
    """문단 확률 0.5로 몰림 방지: 음성은 전체↓, 양성은 top-k만 ↑"""
    if y < 0.5:
        target = torch.zeros_like(s_logits)
        return nn.functional.binary_cross_entropy_with_logits(s_logits, target)
    else:
        k = min(k, s_logits.size(0))
        topk_idx = torch.topk(alpha, k=k, dim=0).indices
        target = torch.zeros_like(s_logits); target[topk_idx] = 1.0
        return nn.functional.binary_cross_entropy_with_logits(s_logits[topk_idx], target[topk_idx])

class AttentionMILModel(nn.Module):
    def __init__(self, encoder: nn.Module, attn_r=128, pos_weight=1.0,
                 pool_mode="lse", lse_tau=10.0, inst_aux_lambda=0.1, inst_aux_k=1):
        super().__init__()
        self.encoder = encoder
        if USE_GC and hasattr(self.encoder, "gradient_checkpointing_enable"):
            self.encoder.gradient_checkpointing_enable()
        if hasattr(self.encoder.config, "use_cache"):
            self.encoder.config.use_cache = False
        d = self.encoder.config.hidden_size
        self.instance_head = nn.Linear(d, 1)
        self.pool = MILPooler(d, mode=pool_mode, r=attn_r, lse_tau=lse_tau)
        self.bag_head = nn.Linear(d, 1)
        # pos_weight(불균형 보정) 등록
        self.register_buffer("pos_weight", torch.tensor(float(pos_weight)))
        # 문단 보조 손실 설정
        self.inst_aux_lambda = inst_aux_lambda
        self.inst_aux_k = inst_aux_k

    def encode_instances(self, input_ids, attention_mask, need_instance_logits=True):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        H = out.last_hidden_state[:, 0, :]                     # [N, d]
        s = self.instance_head(H).squeeze(-1) if need_instance_logits else None
        return H, s

    def forward(self, input_ids, attention_mask, bag_bounds, labels=None,
                return_instance=True, return_alphas=False, **_):
        H_all, s_all = self.encode_instances(input_ids, attention_mask, need_instance_logits=return_instance)
        bag_logits, alphas_all = [], ([] if return_alphas else None)

        inst_reg = 0.0
        for bi, (start, end) in enumerate(bag_bounds.tolist()):
            H = H_all[start:end]
            z, alpha = self.pool(H)
            S = self.bag_head(z).squeeze(-1)
            bag_logits.append(S)
            if return_alphas:
                alphas_all.append(alpha)
            if return_instance and labels is not None:
                s_i = s_all[start:end]
                inst_reg = inst_reg + instance_aux_loss(s_i, alpha, labels[bi], k=self.inst_aux_k)

        bag_logits = torch.stack(bag_logits, dim=0)           # [B]
        out = {"bag_logits": bag_logits}
        if return_instance: out["instance_logits"] = s_all
        if return_alphas:   out["alphas"] = alphas_all

        if labels is not None:
            bce = nn.functional.binary_cross_entropy_with_logits(
                bag_logits, labels, pos_weight=self.pos_weight
            )
            # λ * (bag 평균 보조손실)
            loss = bce + self.inst_aux_lambda * (inst_reg / len(bag_bounds))
            out["loss"] = loss
        return out

# Electra encoder + LoRA(FeatureExtraction)
base_encoder = ElectraModel.from_pretrained(BASE_MODEL)
lora_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.1,
    target_modules=["query","value"],  # 필요시 "key","dense" 추가
    task_type="FEATURE_EXTRACTION"     # 중요: 분류 헤드가 아닌 임베딩 추출용
)
encoder_peft = get_peft_model(base_encoder, lora_cfg)

model = AttentionMILModel(
    encoder=encoder_peft,
    attn_r=128,
    pos_weight=POS_WEIGHT,
    pool_mode=POOL_MODE,     # 'lse'가 attn보다 예리(중간값 완화)
    lse_tau=LSE_TAU,
    inst_aux_lambda=INST_LAMBDA, # 0.05~0.2 추천 범위
    inst_aux_k=INST_TOPK,
).to(device)

if USE_COMPILE and hasattr(torch, "compile"):
    model = torch.compile(model)

print("model ok | encoder:", type(model.encoder).__name__)


pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

model ok | encoder: PeftModelForFeatureExtraction


In [ ]:
# 옵티마/스케줄러 & 안전호출
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

enc_params, head_params = [], []
for n,p in model.named_parameters():
    if any(k in n for k in ["instance_head","pool","bag_head"]):
        head_params.append(p)
    else:
        enc_params.append(p)

optimizer = AdamW([
    {"params": enc_params, "lr": LR_ENCODER},
    {"params": head_params, "lr": LR_HEAD},
], weight_decay=WEIGHT_DECAY)

num_training_steps = EPOCHS * len(tr_loader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=max(1, int(0.1*num_training_steps)),   # 워밍업↑ (MIL 안정화)
    num_training_steps=num_training_steps
)

# BF16이면 GradScaler 불필요
from torch.cuda.amp import GradScaler
scaler = GradScaler(enabled=(not USE_BF16))

def safe_forward(model, batch, *, train=True, return_instance=True, return_alphas=False):
    assert model.__class__.__name__ == "AttentionMILModel"
    return model(
        input_ids      = batch["input_ids"],
        attention_mask = batch["attention_mask"],
        bag_bounds     = batch["bag_bounds"],
        labels         = (batch["labels"] if train else None),
        return_instance= return_instance,
        return_alphas  = return_alphas,
    )

def _gpu_mem():
    if not torch.cuda.is_available(): return "CPU"
    a = torch.cuda.memory_allocated() / (1024**2)
    r = torch.cuda.memory_reserved() / (1024**2)
    return f"{a:.0f}/{r:.0f}MB"


/tmp/ipython-input-3138417607.py:26: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(not USE_BF16))


In [ ]:
from collections import defaultdict
from tqdm.notebook import tqdm

hard_cache = defaultdict(list)

@torch.no_grad()
def refresh_hard_cache_fast(
    model,
    dataset,
    tokenizer,
    max_len_cache=160,     # 캐시 전용 max_len (학습용보다 더 짧게)
    topk=2,                # 항상 포함할 의심 문단 수
    sample_k=64,           # 문서당 캐시 계산에 사용할 최대 문단 수 (부분 샘플)
    pbar=True
):
    """
    각 문서에서 '의심 점수 = alpha * sigmoid(s_i)' 기준 상위 topk 문단 인덱스를 캐시에 저장.
    - 매우 긴 문서는 sample_k개만 랜덤으로 뽑아 빠르게 점수 산출(속도 보장)
    - 진행률 바가 표시됨
    """
    hard_cache.clear()
    model.eval()

    it = dataset.docs
    if pbar:
        it = tqdm(it, total=len(dataset), desc="build hard-cache", ncols=120)

    for doc in it:
        title = doc["title"]
        paras = doc["paras"] if doc["paras"] else [""]

        # 1) 길면 부분 샘플 (원본 인덱스 기억)
        if len(paras) > sample_k:
            sel_idx = np.sort(np.random.choice(len(paras), sample_k, replace=False))
            sel_paras = [paras[i] for i in sel_idx]
        else:
            sel_idx = np.arange(len(paras))
            sel_paras = paras

        # 2) 토큰화 (캐시 전용으로 더 짧은 max_len)
        enc = tokenizer(
            sel_paras, truncation=True, padding=True,
            max_length=max_len_cache, return_tensors="pt"
        )
        batch = {k: v.to(device) for k, v in enc.items()}

        # 3) 한 번의 인퍼런스 (해당 문서만 bag으로 묶음)
        with autocast_ctx():
            out = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                bag_bounds=torch.tensor([(0, len(sel_paras))], dtype=torch.long, device=device),
                labels=None, return_instance=True, return_alphas=True
            )

        # 4) 점수 계산: alpha * sigmoid(s_i)
        s = torch.sigmoid(out["instance_logits"].float()).detach().cpu().numpy()
        a = out["alphas"][0].detach().cpu().numpy()
        score = a * s

        # 5) 상위 topk (부분 샘플 인덱스를 원본 인덱스로 되돌림)
        k = max(1, min(topk, len(sel_idx)))
        local_top = np.argsort(-score)[:k]
        global_top = sel_idx[local_top]
        hard_cache[title] = np.sort(global_top).tolist()

        # 6) 메모리 청소(긴 문서 대비)
        del enc, batch, out
        torch.cuda.empty_cache()
        try:
            torch.cuda.ipc_collect()
        except:
            pass

    model.train()
    print(f"[hard-cache] built for {len(hard_cache)} titles "
          f"(topk={topk}, sample_k={sample_k}, max_len_cache={max_len_cache})")


In [ ]:
#경량검증
@torch.no_grad()
def evaluate_bag_light(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.inference_mode():
        for batch in loader:
            batch = {k:(v.to(device) if torch.is_tensor(v) else v) for k,v in batch.items()}
            with autocast_ctx():
                out = safe_forward(model, batch, train=False, return_instance=False, return_alphas=False)
                logits = out["bag_logits"].float()              # ✅ 안전 캐스팅
                probs  = torch.sigmoid(logits).detach().cpu().numpy()
            ys.extend(batch["labels"].cpu().numpy()); ps.extend(probs)
            del out, batch, logits, probs
            torch.cuda.empty_cache()
            try: torch.cuda.ipc_collect()
            except: pass
    ys = np.array(ys); ps = np.array(ps).reshape(-1)
    return {"AUC": roc_auc_score(ys, ps), "PR_AUC": average_precision_score(ys, ps)}


In [ ]:
# 학습 루프 (BF16/FP16 자동, 서브샘플링으로 빠르게)
best_auc, global_step = -1.0, 0

for ep in range(1, EPOCHS+1):
   # 🔸 에폭 시작 시 캐시 새로고침 (중요!)
    refresh_hard_cache_fast(
    model, tr_set, tokenizer,
    max_len_cache=160,   # 캐시 전용으로 짧게
    topk=HARD_CACHE_TOPK,
    sample_k=64,         # 문서당 캐시 계산에 최대 64문단만 사용
    pbar=True            # 진행률 표시
)
    model.train()
    running = 0.0
    pbar = tqdm(
      total=len(tr_loader),
      desc=f"Epoch {ep}/{EPOCHS}",
      ncols=140,                # ✅ 넓이 확대
      leave=True,
      bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]",
      colour="#4CAF50"          # ✅ 초록색
      )

    for step, batch in enumerate(tr_loader, start=1):
        batch = {k:(v.to(device) if torch.is_tensor(v) else v) for k,v in batch.items()}
        optimizer.zero_grad(set_to_none=True)

        with autocast_ctx():
            out  = safe_forward(model, batch, train=True, return_instance=True, return_alphas=False)
            loss = out["loss"]

        if USE_BF16:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
        else:
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer); scaler.update()

        scheduler.step()
        global_step += 1
        running = loss.item() if step==1 else 0.9*running + 0.1*loss.item()
        pbar.set_postfix({"loss": f"{loss.item():.4f}", "avg": f"{running:.4f}",
                          "lr": f"{scheduler.get_last_lr()[-1]:.2e}", "mem": _gpu_mem()})
        pbar.update(1)
        del out, batch, loss
    pbar.close()

    torch.cuda.empty_cache()
    try: torch.cuda.ipc_collect()
    except: pass

    m = evaluate_bag_light(model, va_loader)
    print(f"[Epoch {ep}] val AUC={m['AUC']:.4f} | PR-AUC={m['PR_AUC']:.4f}")
    if m["AUC"] > best_auc:
        best_auc = m["AUC"]
        torch.save(model.state_dict(), OUTPUT_DIR+"attnmil_lora_best6.pt")
        ckpt = {
          "model": model.state_dict(),
          "optimizer": optimizer.state_dict(),
          "scheduler": scheduler.state_dict(),
          "scaler": scaler.state_dict(),
          "epoch": ep,
          "best_auc": best_auc,
          "lora_cfg": lora_cfg.to_dict(),        # 안전하게 같이 저장
          "base_model": BASE_MODEL,
        }
        torch.save(ckpt, OUTPUT_DIR+"attnmil_full_ckpt6.pt")
        print("  ↳ best saved")


In [ ]:
@torch.no_grad()
def collect_bag_logits(model, loader):
    model.eval()
    logits, labels = [], []
    with torch.inference_mode():
        for batch in loader:
            batch = {k:(v.to(device) if torch.is_tensor(v) else v) for k,v in batch.items()}
            with autocast_ctx():
                out = safe_forward(model, batch, train=False, return_instance=False, return_alphas=False)
                logits.append(out["bag_logits"].detach().cpu())
                labels.append(batch["labels"].detach().cpu())
    return torch.cat(logits), torch.cat(labels)

val_logits, val_labels = collect_bag_logits(model, va_loader)

import torch.nn.functional as F
log_T = torch.nn.Parameter(torch.tensor(0.0, device=device))
opt_T = torch.optim.Adam([log_T], lr=0.05)
val_logits_d, val_labels_d = val_logits.to(device), val_labels.to(device)

best_T, best_loss = None, float("inf")
for _ in range(400):
    opt_T.zero_grad(set_to_none=True)
    T = F.softplus(log_T) + 1e-6
    loss = F.binary_cross_entropy_with_logits(val_logits_d / T, val_labels_d)
    loss.backward()
    opt_T.step()
    if loss.item() < best_loss:
        best_loss, best_T = loss.item(), float(T.item())
print(f"✅ Optimized Temperature T = {best_T:.4f}")


model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

✅ Optimized Temperature T = 0.0385


In [ ]:
test_df = pd.read_csv(DIR+"test.csv")
assert set(["ID","title","paragraph_index","paragraph_text"]).issubset(test_df.columns)
test_df["paragraph_text"] = test_df["paragraph_text"].astype(str).fillna("")
test_df = test_df.sort_values(["title","paragraph_index"]).reset_index(drop=True)

class TestParagraphDataset(Dataset):
    def __init__(self, df):
        self.groups = []
        for title, g in df.groupby("title"):
            self.groups.append({"title": title, "paras": g["paragraph_text"].tolist(), "index": g.index.tolist()})
    def __len__(self): return len(self.groups)
    def __getitem__(self, idx): return self.groups[idx]

def collate_test(batch, tokenizer, max_len=256):
    all_texts, bag_bounds, titles, indices = [], [], [], []
    cursor = 0
    for d in batch:
        titles.append(d["title"]); indices.append(d["index"])
        paras = d["paras"] if d["paras"] else [""]
        start = cursor
        all_texts.extend(paras)
        cursor += len(paras); end = cursor
        bag_bounds.append((start, end))
    enc = tokenizer(all_texts, truncation=True, padding=True, max_length=max_len, return_tensors="pt")
    return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"],
            "bag_bounds": torch.tensor(bag_bounds, dtype=torch.long),
            "titles": titles, "indices": indices}

te_set = TestParagraphDataset(test_df)
te_loader = DataLoader(te_set, batch_size=1, shuffle=False,
                       collate_fn=lambda b: collate_test(b, tokenizer, MAX_LEN))

# 동일 구조로 best 로드
best_encoder = ElectraModel.from_pretrained(BASE_MODEL)
best_encoder = get_peft_model(best_encoder, lora_cfg)
best_model = AttentionMILModel(
    encoder=best_encoder, attn_r=128, pos_weight=POS_WEIGHT,
    pool_mode="lse", lse_tau=10.0, inst_aux_lambda=0.1, inst_aux_k=1
).to(device)
best_model.load_state_dict(torch.load(OUTPUT_DIR+"attnmil_lora_best6.pt", map_location=device))
best_model.eval()

@torch.no_grad()
def infer(model, loader, T: float):
    paragraph_probs = np.zeros(len(test_df), dtype=float)
    model.eval()
    with torch.inference_mode():
        for batch in loader:
            batch = {k: (v.to(device) if torch.is_tensor(v) else v) for k, v in batch.items()}
            with autocast_ctx():
                out = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    bag_bounds=batch["bag_bounds"],
                    labels=None,
                    return_instance=True,
                    return_alphas=False,
                )

                # ✅ 문단 로짓을 float32로 캐스팅 후 sigmoid
                inst_logits = out["instance_logits"].float()               # <-- 핵심 1
                p_all = torch.sigmoid(inst_logits).detach().cpu().numpy() # <-- 핵심 2

                # (문서 확률 보고용이 필요하면)
                # doc_prob = torch.sigmoid(out["bag_logits"].float() / T).item()

                (start, end) = batch["bag_bounds"][0].tolist()
                idxs = batch["indices"][0]
                paragraph_probs[idxs] = p_all[start:end]

            del out, batch, inst_logits, p_all
            torch.cuda.empty_cache()
            try:
                torch.cuda.ipc_collect()
            except:
                pass
    return paragraph_probs

paragraph_probs = infer(best_model, te_loader, T=best_T)

submission = pd.DataFrame({"ID": test_df["ID"], "generated": paragraph_probs})
# 방법 1) 보조 컬럼 만들어 정렬
submission["id_num"] = submission["ID"].str.extract(r"(\d+)").astype(int)
submission = submission.sort_values("id_num").drop(columns="id_num").reset_index(drop=True)
submission.to_csv(OUTPUT_DIR+"submission_fast_strong6.csv", index=False)
print("Saved:", submission.shape, "→ submission_fast_strong6.csv")


Saved: (1962, 2) → submission_fast_strong6.csv
